In [1]:
import numpy as np
import os
from sklearn.externals import joblib
import pandas as pd
from Bio import SeqIO
import pandas as pd
from Bio.Alphabet import IUPAC

In [2]:
ogt_file = '../Brenda_Annotation/2_unid_growth_temp_mapping.tsv'
seq_file = '../BrendaData_20180109/brenda_seqeunces_20180109.fasta'

In [3]:
# load ogt
dfogt = pd.read_csv(ogt_file,sep='\t')
ogts = dict()
for ind in dfogt.index: ogts[dfogt.loc[ind,'uniprot_id']] = dfogt.loc[ind,'growth_temp']

In [4]:
# 2. extract kmer features
def get_kmer_fq(seq):
    fea = dict()
    seq = seq.upper()
    aas = IUPAC.IUPACProtein.letters
    count = dict()

    for a in aas: count[a] = 0
    for a in seq: 
        try: count[a] += 1
        except: None
    
    L = sum(count.values())
    L = float(L)
    for a,v in count.items(): fea[a] = v/L
    return fea

In [5]:
data = list()
uni_ids = list()

aas = list(IUPAC.IUPACProtein.letters)
num = 1
for rec in SeqIO.parse(seq_file,'fasta'):
    uni = rec.id
    ogt = ogts.get(uni,None)
    
    if ogt is None: continue
    
    uni_ids.append(uni)
    seq = str(rec.seq)
    fq = get_kmer_fq(seq)
    
    x = [fq[a] for a in aas]
    x.append(ogt)
    data.append(x)
    num += 1
    if num%100000 == 0: print num
data = np.array(data)
df = pd.DataFrame(data=data,index=uni_ids,columns=aas+['OGT'])
print data.shape

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
(6270107, 21)


In [6]:
# 1. load the trained model for prediction
predictor='../model_v3/predictor/Topt_RF.pkl'
model=joblib.load(predictor)

def get_means_vars(predictor):
    means=dict()
    vs=dict()
    features=list()
    for line in open(predictor.replace('pkl','f'),'r'):
        if line.startswith('#'):continue
        cont=line.split()
        means[cont[0]]=float(cont[1])
        vs[cont[0]]=float(cont[2])
        features.append(cont[0])
    return means,vs,features

means,vs,features=get_means_vars(predictor)

/c3se/users/gangl/Hebbe/Tools/my_python_new/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
# 2. do norm
dfnew = df[features]
X = dfnew.values
for i in range(len(features)):
    print i
    fea = features[i]
    X[:,i] = (X[:,i]-means[fea])/vs[fea]

# Predict
Ypre = model.predict(X)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [8]:
# save predicted results
outfile = '../Brenda_Annotation/predicted_topt.tsv'
fhand = open(outfile,'w')
fhand.write('ID\tOGT\tTopt\n')
for i in range(X.shape[0]): fhand.write('{0}\t{1}\t{2}\n'.format(dfnew.index[i],df['OGT'][i],Ypre[i]))
fhand.close()